In [1]:
import csv
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly
import os
import tqdm
import json
from collections import defaultdict
import json_lines
import datetime
import matplotlib.pyplot as plt
from statistics import median
from ipywidgets import interact

# 파일 불러오기

In [2]:
# 성지 후보군 기사들
with open('../pilgrim data/result/current_pilgrim_candidates.json') as f:
    cands = json.load(f)

# sid 별 매칭 정보가 담긴 파일
with open('../pilgrim data/sid_to_cat.csv', 'r', encoding = 'utf-8') as g:
    reader = csv.reader(g)
    sid_to_cat = {line[0]: line[1] for line in reader}
    del sid_to_cat['\ufeffsid']

In [9]:
cands = sorted(cands.items(), key = lambda x: x[1]['count'], reverse = True)

In [10]:
i = 1
for k,v in cands:
    print('{}. {}'.format(i, v['title']))
    print('URL: {}'.format(v['url']))
    i += 1

1. “박수진 삼성병원 특혜 조사해주세요”, 청와대 청원 5만명 돌파
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=117&aid=0002988452
2. [단독] 최태원 내연녀 김 씨, 어머니 대신 금감원 출석시켜
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=056&aid=0010276883
3. 安 "청년, 투표 안하니 지원법 저조…어느 당 찍든 투표해야"
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0007125210
4. '낙태강요' 동거남 부친 살해 30대女 징역 30년 확정
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=422&aid=0000222340
5. "'W' OST 불렀쓰요"…정준영-한효주, 특급 만남
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=076&aid=0002963289
6. 더민주 '방산 비리' 이적죄로 규정…사형·무기징역 가능토록
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0002095013
7. 생수 마신 5세 아이까지 피폭, 더는 살 수 없다
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=047&aid=0002106177
8. ‘핀테크’ 집중 육성…3년간 3조원 지원…전자화폐 ‘비트코인’ 제도권 편입한다
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101

# 성지 후보 목록의 기사들을 카테고리 별로 분류

In [11]:
pilg_categories = defaultdict(int)
for k,v in cands:
    data = v
    cats = []
    for sid in data['sid1s']:
        cats.append(sid_to_cat[sid])
    pilg_categories[','.join(cats)] += 1

ordered_cat =  sorted(
    list(pilg_categories.items()), key=lambda x: x[1], reverse=True)
ordered_cat

[('사회', 72),
 ('정치', 37),
 ('연예', 31),
 ('생활/문화', 16),
 ('경제', 9),
 ('분류안됨', 5),
 ('사회,생활/문화', 2),
 ('세계', 2),
 ('오피니언', 1),
 ('정치,사회,생활/문화', 1),
 ('정치,경제', 1)]

# Timeline CDF로 각 성지를 표현해보기
## 직접적인 성지 언급이 있는 기사: 174건

### 각 뉴스 별 CDF 확인

In [14]:
pilg_by_cat = defaultdict(list)
for cand in cands:
    data = cand[1]
    cats = []
    for sid in data['sid1s']:
        cats.append(sid_to_cat[sid])
    pilg_by_cat[','.join(cats)].append(data)

        
@interact (idx = (0, len(cands)))
def show_cdf (idx = 0):
    fig = go.Figure()
    target = cands[idx][1]
    
    #기사정보 표시하기
    print('Title: {}'.format(target['title']))
    print('Published: {}'.format(target['timestamp']))
    print('URL: {}'.format(target['url']))
    
    timeline = target['timeline']
#     for k,v in timeline.items():
#         print('{} : {} comments'.format(k,v))
    
    published_date = datetime.datetime.strptime(target['timestamp'][:10], '%Y-%m-%d')
    dates = list(map(lambda x: datetime.datetime.strptime(x, '%Y%m%d'), list(target['timeline'].keys())))
    cnts = np.cumsum(list(target['timeline'].values())) / sum(list(target['timeline'].values()))
    dates.insert(0, published_date)
    cnts = np.insert(cnts, 0, 0)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dates, y = cnts, mode='markers+lines'))
    fig.update_yaxes(range = [0,1])
    # fig.update_xaxes(range=[published_date,
    #                                datetime.datetime(2019, 12, 31)])
    fig.update_xaxes(range=[published_date,
                                   dates[-1]])
    fig.show()

interactive(children=(IntSlider(value=0, description='idx', max=177), Output()), _dom_classes=('widget-interac…

### 카테고리 별로 CDF 묶어서 확인.

In [17]:
@interact (idx = (0,len(pilg_by_cat)))
def show_cdf_by_category (idx = 0):
    fig = go.Figure()
    targets = pilg_by_cat[ordered_cat[idx][0]]
    
    for target in targets:
        timeline = target['timeline']
        published_date = datetime.datetime.strptime(target['timestamp'][:10], '%Y-%m-%d')
        dates = list(map(lambda x: datetime.datetime.strptime(x, '%Y%m%d'), list(target['timeline'].keys())))
        cnts = np.cumsum(list(target['timeline'].values())) / sum(list(target['timeline'].values()))
        dates.insert(0, published_date)
        cnts = np.insert(cnts, 0, 0)
        fig.add_trace(go.Scatter(x = dates, y = cnts, mode='markers+lines', name = target['title']))
    
    fig.update_yaxes(range = [0,1])
    fig.update_xaxes(range=[datetime.datetime(2015,12,1),
                                   datetime.datetime(2019,12,31)])
    fig.update_layout(title = "카테고리: {} / 기사 수: {}개".format(ordered_cat[idx][0], ordered_cat[idx][1]))
    fig.show()

interactive(children=(IntSlider(value=0, description='idx', max=11), Output()), _dom_classes=('widget-interact…

# Pilgrim Score
- 발행 이후 N 일 뒤의 댓글 수가 많을수록
- 댓글이 달린 기간이 길수록
- 최근에 달린 댓글이 많을수록
- 단발성, 집중성 성지가 아닐 수록

    더 좋은 성지에 가깝도록 점수를 구체화.
    
3개의 스코어를 만들어 그것을 합산하여 각 기사 별 점수를 뽑아내도록 함.

- OLD_SCORE
    - 각 댓글이 기사 생성일로부터 오래 될수록 가중치를 부여하여 매기는 점수
    - 각 기사 별로, 1년 이상된 댓글들의 평균 나이를 계산.
    
- RECENT_SCORE
    - 최근에 작성된 댓글로 이루어진 성지일수록 높은 점수를 받도록 구성. 
    - 같은 달에 쓰여진 댓글의 경우 같은 가중치를 갖도록. (12월 30일과 12월 1일은 거시적인 관점에서 큰 차이 없게.)
    
- CONCENTRATED_SCORE
    - 특정 기간에 집중되어 만들어진 성지를 찾고자 함.
    - 이게 너무 높으면 좋은 성지라고 보기는 어렵도록 설정.
    - 이 값이 클 때에만 전체 스코어에 큰 영향을 끼치도록, 작을 수록 별 차이 없도록 함.
    - 이 값이 특정 값 이상일 경우 전부 같은 값을 갖도록 함.
    - 레벨화하여 같은 레벨에서는 소수점 값이 같더라도 같은 영향력을 가지도록 함.

In [18]:
def total_score(info):
    '''
    input: 각 기사에 대한 정보 (timeline, timestamp, etc)
    output: 각 기사 별 pilgrim score

    OLD_SCORE: 각 댓글이 기사 생성일로부터 오래 될수록 가중치를 부여하여 매기는 점수
    RECENT_SCORE: 최근에 작성된 댓글로 이루어진 성지일수록 높은 점수를 받도록 구성.
                  같은 달에 쓰여진 댓글의 경우 같은 가중치를 갖도록. (12월 30일과 12월 1일은 거시적인 관점에서 큰 차이 없게.)
    CONCENTRATED_SCORE: 특정 기간에 집중되어 만들어진 성지를 찾고자 함.
                        이게 너무 높으면 좋은 성지라고 보기는 어렵도록 설정.
                        이 값이 클 때에만 전체 스코어에 큰 영향을 끼치도록, 작을 수록 별 차이 없도록 최종 식에서 설정.
    '''
    N = 365
    published_date = datetime.datetime.strptime(
        info['timestamp'][:10], '%Y-%m-%d')
    timeline = info['timeline']
    dates = list(map(lambda x: (x, datetime.datetime.strptime(
        x, '%Y%m%d')), list(timeline.keys())))
    length = (dates[-1][1] - published_date).days

    old_score = 0
    recent_score = 0
    days = []
    pilg_count = 0

    for string, date in dates:
        diff = (date - published_date).days
        if diff >= N:
            old_score += diff * timeline[string]
            recent_score += 1 / \
                ((datetime.datetime(2019, 12, 31) -
                  date).days // 30) * timeline[string]
            pilg_count += timeline[string]
            for i in range(timeline[string]):
                days.append(diff)

    os = old_score / pilg_count / 365
    rs = recent_score / pilg_count * 100  # 절대적인 크기를 맞추어주기 위해서
    amount = len(days)
    # 집중도를 계산하는 데에 존재하는 아웃라이어들을 제거.
    days = days[5:-
                5] if amount > 50 else days[int(amount * .1): int(amount * .9)]
    cs = 10e-3 if np.std(days) == 0 else np.std(days)
    cs = 5 if 1/cs > 5 else 1/cs
    cs = int(cs+1)

    return os, rs, cs, (os * rs) / cs * pilg_count
#     return os, rs, cs, (os * rs) * cs * pilg_count


def num_pilg_comments(info):
    N = 365
    published_date = datetime.datetime.strptime(
        info['timestamp'][:10], '%Y-%m-%d')
    timeline = info['timeline']
    dates = list(map(lambda x: (x, datetime.datetime.strptime(
        x, '%Y%m%d')), list(timeline.keys())))
    length = (dates[-1][1] - published_date).days
    pilg_count = 0
    for string, date in dates:
        diff = (date - published_date).days
        if diff >= N:
            pilg_count += timeline[string]

    return pilg_count

## Pilgrim Score 순서로 기사 정렬

In [20]:
infos = [cand[1] for cand in cands]
scored_candidates = sorted(list(map(lambda x: (total_score(x), num_pilg_comments(x),
                                               x['title'], x['url']), infos)),
                          key = lambda x: x[0][3], reverse = True)

i = 1
for (os, rs, cs, ts), num, title, url in scored_candidates:
    print('{}. {}'.format(i, title))
    print('num: {}  old: {:.3f}yr  recent: {:.3f}  tight_level: {}'.format(num, os, rs, cs))
    print('score: {:.3f}'.format(ts))
    print('URL: {}'.format(url))
    i += 1

1. “박수진 삼성병원 특혜 조사해주세요”, 청와대 청원 5만명 돌파
num: 2219  old: 1.271yr  recent: 11.631  tight_level: 1
score: 32802.482
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=117&aid=0002988452
2. '낙태강요' 동거남 부친 살해 30대女 징역 30년 확정
num: 575  old: 1.975yr  recent: 8.054  tight_level: 1
score: 9144.015
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=422&aid=0000222340
3. [단독] 최태원 내연녀 김 씨, 어머니 대신 금감원 출석시켜
num: 971  old: 1.410yr  recent: 3.320  tight_level: 1
score: 4544.553
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=056&aid=0010276883
4. “혹시 한남?”… 소개팅 성공 기준된 페미니즘
num: 223  old: 1.363yr  recent: 12.844  tight_level: 1
score: 3903.058
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=469&aid=0000258938
5. "'W' OST 불렀쓰요"…정준영-한효주, 특급 만남
num: 572  old: 2.775yr  recent: 14.214  tight_level: 6
score: 3760.534
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=076&aid=0002963289
6. 安 "청년, 투표 안하니 

# 번외

## (가장 최근 댓글) - (발행일) 순서대로 정렬

In [23]:
# 최장기간 성지 찾기
def lifetime (info):
    published_date = datetime.datetime.strptime(info['timestamp'][:10], '%Y-%m-%d')
    timeline = info['timeline']
    dates = list(map(lambda x: (x, datetime.datetime.strptime(x, '%Y%m%d')), list(timeline.keys())))
    length = (dates[-1][1] - published_date).days
    
    return length
    
infos = [cand[1] for cand in cands]
scored_candidates = sorted(list(map(lambda x: (lifetime(x), x['title'], x['url']), infos)),
                          key = lambda x: x[0], reverse = True)

i = 1
for score, title, url in scored_candidates:
    print('{}. {}'.format(i, title))
    print('기간: {}년 {}일'.format(score//365, score%365))
    print('URL: {}'.format(url))
    i += 1

1. “속옷 차림 글래머 여군이라니”… 뷰티풀 군바리 피규어 눈살
기간: 3년 180일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=005&aid=0000852380
2. 인권위 “환경미화원 채용에 남녀 동일한 체력시험, 성차별이다”
기간: 3년 120일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=016&aid=0000981887
3. 부산 문현동 '금도굴' 사건이란?
기간: 3년 108일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=079&aid=0002784325
4. 방탄소년단 지민&돈 스파이크, 도플갱어설 휘말려
기간: 3년 93일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=052&aid=0000782020
5. 무기력한 경찰…취객에 테이저건 빼앗기고 폭행당해(종합)
기간: 3년 71일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0008265061
6. 2. 공포의 운전대
기간: 3년 57일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=214&aid=0000583800
7. 또 데이트폭력, 결혼 안 해준다고 손가락 잘라
기간: 3년 53일
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=214&aid=0000606004
8. '고통없이 죽는 약' 마약 밀반입한 30대 징역형
기간: 3년 9일
URL: https://news.naver.com/main/read.

## 총 댓글 수 순서대로 정렬

In [25]:
infos = [cand[1] for cand in cands]
scored_candidates = sorted(list(map(lambda x: (sum(list(x['timeline'].values())), x['title'], x['url']), infos)),
                          key = lambda x: x[0], reverse = True)

i = 1
for score, title, url in scored_candidates:
    print('{}. {}'.format(i, title))
    print('댓글 수: {}개'.format(score))
    print('URL: {}'.format(url))
    i += 1

1. "보.이.콧"…제이에스티나, 中에서 송혜교 역풍
댓글 수: 127531개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=433&aid=0000016711
2. “박수진 삼성병원 특혜 조사해주세요”, 청와대 청원 5만명 돌파
댓글 수: 23769개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=117&aid=0002988452
3. [단독] 최태원 내연녀 김 씨, 어머니 대신 금감원 출석시켜
댓글 수: 14486개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=056&aid=0010276883
4. 슈주 성민, 팬 활동중지요구→사과..'소문'과 '변론' 사이
댓글 수: 11004개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=108&aid=0002628475
5. [속보]교육부, “민중은 개돼지” 나향욱 정책기획관 ‘파면’ 결정
댓글 수: 8551개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=032&aid=0002713605
6. '조현병' 소녀에게 살인의 책임을 물을 수 있나
댓글 수: 6592개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=008&aid=0003853354
7. [하나님의교회 세계복음선교협회]영국여왕, 하나님의 교회에 최고 영예 ‘여왕상’ 수여
댓글 수: 6539개
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=020&aid=0002988438
8. 박해진, 웃음·